In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext line_profiler

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../'))
from api import API
from data import Data
import pandas as pd

## Closest Matching UG Playlist to an ED playlist

In [3]:
d = Data()

In [23]:
diffs, ugPIDs = d.closestMatchingUGPlaylist("Are & Be")

In [25]:
ugPIDs[0]

213590

In [26]:
diffs[0]

1.466596422333839

In [27]:
names, artists = d.getUGPlaylist(213590)

In [28]:
names

['Tiny Little Robots',
 'Shoot to Thrill',
 'Immigrant Song',
 'Welcome to Paradise',
 'Hysteria',
 'Supermassive Black Hole',
 'Uprising',
 "The Kids Aren't Alright",
 'Bad Habit',
 'Coming for You',
 'Tom Sawyer',
 'Brianstorm',
 'N.I.B.',
 'Children of the Grave - Remastered Version',
 "I'm Shipping Up To Boston",
 'The State Of Massachusetts',
 'Paradise City',
 'Welcome To The Jungle',
 'For Whom The Bell Tolls - Remastered',
 'A Moment of Violence',
 'Everything Went Numb',
 'Icky Thump',
 'Ace of Spades',
 'Come Out And Play (Keep ‘Em Separated)',
 'Enter Sandman',
 'The Trooper - 1998 Remastered Version',
 'Hallowed Be Thy Name - 1998 Remastered Version',
 'Holy Wars...The Punishment Due - 2004 Digital Remaster',
 'Black Dragon',
 'Collapse (Post-Amerika)',
 'Woman']

In [29]:
artists

['Cage The Elephant',
 'AC/DC',
 'Led Zeppelin',
 'Green Day',
 'Muse',
 'Muse',
 'Muse',
 'The Offspring',
 'The Offspring',
 'The Offspring',
 'Rush',
 'Arctic Monkeys',
 'Black Sabbath',
 'Black Sabbath',
 'Dropkick Murphys',
 'Dropkick Murphys',
 "Guns N' Roses",
 "Guns N' Roses",
 'Metallica',
 'Streetlight Manifesto',
 'Streetlight Manifesto',
 'The White Stripes',
 'Motörhead',
 'The Offspring',
 'Metallica',
 'Iron Maiden',
 'Iron Maiden',
 'Megadeth',
 'The Vines',
 'Rise Against',
 'Wolfmother']

## Predicting playlist avg features from playlist names

In [4]:
features, names = d.getUGPlaylistFeaturesAndNames()

In [5]:
len(features)

1000000

In [6]:
features[:50]

[[0.7822692307692307,
  0.6592115384615383,
  5.038461538461538,
  -4.8784807692307695,
  0.6923076923076923,
  0.10718846153846154,
  0.08298072884615385,
  0.00067601,
  0.19113846153846153,
  0.6426153846153846,
  123.00753846153846,
  4.0],
 [0.690128205128205,
  0.4947153846153846,
  4.461538461538462,
  -8.288435897435898,
  0.5384615384615384,
  0.08908461538461539,
  0.16404996025641028,
  0.2244309776923077,
  0.18570769230769232,
  0.4754871794871795,
  124.90761538461538,
  3.769230769230769],
 [0.693203125,
  0.671875,
  5.0,
  -4.87415625,
  0.515625,
  0.0962875,
  0.26922984375,
  0.0006378143749999999,
  0.169028125,
  0.565078125,
  114.600671875,
  4.0],
 [0.6212420634920635,
  0.513563492063492,
  5.103174603174603,
  -9.614833333333333,
  0.7142857142857143,
  0.0672015873015873,
  0.27384541428571424,
  0.20205686214285715,
  0.18883412698412697,
  0.45190079365079366,
  125.03249206349206,
  3.9523809523809526],
 [0.6505352941176471,
  0.576764705882353,
  3.35294

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, Activation
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.metrics import mean_squared_error
import re
from sklearn.preprocessing import MinMaxScaler

C:\Users\Adam Khazi\AppData\Local\conda\conda\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
namesPD = pd.DataFrame(names, columns=['text'])

In [9]:
scaler = MinMaxScaler()
scaledFeatures = scaler.fit_transform(features)

In [10]:
scaledFeatures[0].shape

(12,)

In [11]:
namesPD.iloc[1000]

text    disney
Name: 1000, dtype: object

In [12]:
namesPD['text'] = namesPD['text'].apply(lambda x: x.lower())
#namesPD['text'] = namesPD['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [13]:
max_features = 1750000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(namesPD['text'].values)
X = tokenizer.texts_to_sequences(namesPD['text'].values)
X = pad_sequences(X)

In [14]:
X[100000]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       1319])

In [15]:
scaledFeatures[0]

array([7.82640881e-01, 6.97397663e-01, 4.83474137e-01, 9.15080042e-01,
       6.92307692e-01, 1.00018388e-01, 8.33883723e-02, 7.05647182e-04,
       1.74221768e-01, 6.56053906e-01, 6.11817686e-01, 8.63636364e-01])

In [16]:
embed_dim = 128
lstm_out = 150

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
#model.add(SpatialDropout1D(0.4))
model.add(Flatten())
#model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(lstm_out))
model.add(Activation('relu'))
model.add(Dense(lstm_out))
model.add(Activation('relu'))
model.add(Dense(12))
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['mse'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 128)           224000000 
_________________________________________________________________
flatten_1 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 150)               230550    
_________________________________________________________________
activation_1 (Activation)    (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 150)               22650     
_________________________________________________________________
activation_2 (Activation)    (None, 150)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 12)                1812      
Total para

In [17]:
scaledFeatures.shape

(1000000, 12)

In [18]:
X.shape

(1000000, 12)

In [93]:
Y = scaledFeatures
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(670000, 12) (670000, 12)
(330000, 12) (330000, 12)


In [94]:
batch_size = 4092
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 2)

Epoch 1/3
 - 334s - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 2/3
 - 333s - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 3/3
 - 330s - loss: 0.0067 - mean_squared_error: 0.0067


In [95]:
validation_size = 15000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.5f" % (score))
print("acc: %.5f" % (acc))

score: 0.00689
acc: 0.00689


In [96]:
Y_preds = model.predict(X_test)

In [97]:
Y_test

array([[0.72374213, 0.57470808, 0.60019268, ..., 0.43706233, 0.61944413,
        0.85653409],
       [0.13596843, 0.24215165, 0.46914315, ..., 0.09094576, 0.48358304,
        0.78676471],
       [0.60505542, 0.77872767, 0.50509221, ..., 0.43259559, 0.63884413,
        0.85974026],
       ...,
       [0.65427118, 0.46511391, 0.43848059, ..., 0.25905563, 0.66285239,
        0.83116883],
       [0.49796627, 0.69586045, 0.56693275, ..., 0.46847224, 0.50328583,
        0.85281385],
       [0.73375751, 0.58773347, 0.49459019, ..., 0.53633612, 0.6524755 ,
        0.8548951 ]])

In [98]:
Y_preds

array([[0.77798283, 0.5673015 , 0.49415705, ..., 0.43877766, 0.6424809 ,
        0.85201263],
       [0.45295024, 0.51846844, 0.4845938 , ..., 0.3250734 , 0.5748331 ,
        0.82712257],
       [0.67283577, 0.64353627, 0.49740967, ..., 0.5144211 , 0.60571235,
        0.85380995],
       ...,
       [0.5593959 , 0.49518394, 0.5015974 , ..., 0.3452358 , 0.5977248 ,
        0.8335781 ],
       [0.5125575 , 0.62125725, 0.50065815, ..., 0.4239278 , 0.5733955 ,
        0.84356   ],
       [0.7092453 , 0.6092514 , 0.5204007 , ..., 0.5805804 , 0.6392689 ,
        0.8519362 ]], dtype=float32)

In [89]:
Y_test.mean()

0.4829267819650614

In [99]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, Y_preds)

0.006894669521572548

## Playlist Track Classifier

In [4]:
import numpy as np

In [115]:
posFeatures, negFeatures = d.getPlaylistWTrackVectorsRefined(10000, 2)









  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]







  0%|                                                                                | 1/10000 [00:00<47:06,  3.54it/s]







  0%|                                                                                | 2/10000 [00:00<33:47,  4.93it/s]







  0%|                                                                                | 3/10000 [00:00<33:51,  4.92it/s]







  0%|                                                                                | 4/10000 [00:00<40:16,  4.14it/s]







  0%|                                                                                | 6/10000 [00:01<34:05,  4.89it/s]







  0%|                                                                                | 8/10000 [00:01<29:35,  5.63it/s]







  0%|                                                                                | 9/10000 [00:01<2

  2%|█▏                                                                            | 157/10000 [00:12<13:15, 12.37it/s]







  2%|█▏                                                                            | 160/10000 [00:12<13:07, 12.50it/s]







  2%|█▎                                                                            | 163/10000 [00:12<12:58, 12.63it/s]







  2%|█▎                                                                            | 167/10000 [00:13<12:45, 12.84it/s]







  2%|█▎                                                                            | 170/10000 [00:13<12:42, 12.89it/s]







  2%|█▎                                                                            | 174/10000 [00:13<12:32, 13.07it/s]







  2%|█▍                                                                            | 179/10000 [00:13<12:16, 13.33it/s]







  2%|█▍                                                                            | 183/10000 [00:13<12:07, 13

  5%|███▌                                                                          | 458/10000 [00:21<07:31, 21.13it/s]







  5%|███▌                                                                          | 463/10000 [00:21<07:28, 21.25it/s]







  5%|███▋                                                                          | 468/10000 [00:21<07:26, 21.33it/s]







  5%|███▋                                                                          | 472/10000 [00:22<07:26, 21.33it/s]







  5%|███▋                                                                          | 476/10000 [00:22<07:25, 21.37it/s]







  5%|███▊                                                                          | 484/10000 [00:22<07:21, 21.57it/s]







  5%|███▊                                                                          | 492/10000 [00:22<07:16, 21.78it/s]







  5%|███▉                                                                          | 500/10000 [00:22<07:11, 22

  9%|██████▊                                                                       | 873/10000 [00:30<05:23, 28.20it/s]







  9%|██████▊                                                                       | 879/10000 [00:31<05:22, 28.29it/s]







  9%|██████▉                                                                       | 885/10000 [00:31<05:21, 28.39it/s]







  9%|██████▉                                                                       | 891/10000 [00:31<05:19, 28.48it/s]







  9%|██████▉                                                                       | 897/10000 [00:31<05:18, 28.57it/s]







  9%|███████                                                                       | 904/10000 [00:31<05:16, 28.70it/s]







  9%|███████                                                                       | 913/10000 [00:31<05:14, 28.90it/s]







  9%|███████▏                                                                      | 920/10000 [00:31<05:13, 28

 14%|██████████▌                                                                  | 1364/10000 [00:39<04:07, 34.95it/s]







 14%|██████████▌                                                                  | 1372/10000 [00:39<04:06, 34.95it/s]







 14%|██████████▌                                                                  | 1379/10000 [00:39<04:06, 35.03it/s]







 14%|██████████▋                                                                  | 1386/10000 [00:39<04:05, 35.07it/s]







 14%|██████████▋                                                                  | 1392/10000 [00:39<04:05, 35.11it/s]







 14%|██████████▊                                                                  | 1403/10000 [00:39<04:03, 35.29it/s]







 14%|██████████▊                                                                  | 1410/10000 [00:39<04:03, 35.27it/s]







 14%|██████████▉                                                                  | 1420/10000 [00:40<04:02, 35

...

 23%|█████████████████▏                                                          | 2261/10000 [00:10<00:34, 223.12it/s]







 23%|█████████████████▍                                                          | 2288/10000 [00:10<00:34, 223.45it/s]







 23%|█████████████████▌                                                          | 2315/10000 [00:10<00:34, 223.86it/s]







 23%|█████████████████▊                                                          | 2343/10000 [00:10<00:34, 224.35it/s]







 24%|██████████████████                                                          | 2370/10000 [00:10<00:33, 224.77it/s]







 24%|██████████████████▏                                                         | 2397/10000 [00:10<00:33, 225.19it/s]







 24%|██████████████████▍                                                         | 2426/10000 [00:10<00:33, 225.74it/s]







 25%|██████████████████▋                                                         | 2454/10000 [00:10<00:33, 226

 41%|██████████████████████████████▉                                             | 4070/10000 [00:16<00:24, 241.58it/s]







 41%|███████████████████████████████▏                                            | 4096/10000 [00:16<00:24, 241.61it/s]







 41%|███████████████████████████████▎                                            | 4122/10000 [00:17<00:24, 241.55it/s]







 42%|███████████████████████████████▌                                            | 4151/10000 [00:17<00:24, 241.78it/s]







 42%|███████████████████████████████▋                                            | 4177/10000 [00:17<00:24, 241.82it/s]







 42%|███████████████████████████████▉                                            | 4208/10000 [00:17<00:23, 242.19it/s]







 42%|████████████████████████████████▏                                           | 4239/10000 [00:17<00:23, 242.55it/s]







 43%|████████████████████████████████▍                                           | 4269/10000 [00:17<00:23, 242

 59%|████████████████████████████████████████████▉                               | 5916/10000 [00:23<00:16, 250.61it/s]







 59%|█████████████████████████████████████████████▏                              | 5945/10000 [00:23<00:16, 250.71it/s]







 60%|█████████████████████████████████████████████▍                              | 5974/10000 [00:23<00:16, 250.77it/s]







 60%|█████████████████████████████████████████████▌                              | 6002/10000 [00:23<00:15, 250.83it/s]







 60%|█████████████████████████████████████████████▊                              | 6032/10000 [00:24<00:15, 251.00it/s]







 61%|██████████████████████████████████████████████                              | 6061/10000 [00:24<00:15, 251.14it/s]







 61%|██████████████████████████████████████████████▎                             | 6090/10000 [00:24<00:15, 251.23it/s]







 61%|██████████████████████████████████████████████▌                             | 6120/10000 [00:24<00:15, 251

 78%|███████████████████████████████████████████████████████████▏                | 7789/10000 [00:30<00:08, 256.29it/s]







 78%|███████████████████████████████████████████████████████████▍                | 7817/10000 [00:30<00:08, 256.25it/s]







 78%|███████████████████████████████████████████████████████████▋                | 7848/10000 [00:30<00:08, 256.39it/s]







 79%|███████████████████████████████████████████████████████████▊                | 7876/10000 [00:30<00:08, 256.46it/s]







 79%|████████████████████████████████████████████████████████████                | 7906/10000 [00:30<00:08, 256.58it/s]







 79%|████████████████████████████████████████████████████████████▎               | 7935/10000 [00:30<00:08, 256.67it/s]







 80%|████████████████████████████████████████████████████████████▌               | 7964/10000 [00:31<00:07, 256.70it/s]







 80%|████████████████████████████████████████████████████████████▋               | 7993/10000 [00:31<00:07, 256

 96%|████████████████████████████████████████████████████████████████████████▊   | 9576/10000 [00:37<00:01, 257.45it/s]







 96%|████████████████████████████████████████████████████████████████████████▉   | 9604/10000 [00:37<00:01, 257.48it/s]







 96%|█████████████████████████████████████████████████████████████████████████▏  | 9633/10000 [00:37<00:01, 257.54it/s]







 97%|█████████████████████████████████████████████████████████████████████████▍  | 9661/10000 [00:37<00:01, 257.55it/s]







 97%|█████████████████████████████████████████████████████████████████████████▋  | 9688/10000 [00:37<00:01, 257.57it/s]







 97%|█████████████████████████████████████████████████████████████████████████▊  | 9715/10000 [00:37<00:01, 257.39it/s]







 97%|██████████████████████████████████████████████████████████████████████████  | 9741/10000 [00:37<00:01, 257.39it/s]







 98%|██████████████████████████████████████████████████████████████████████████▏ | 9767/10000 [00:37<00:00, 257

In [ ]:
%lprun -f d.getPlaylistWTrackVectorsRefined d.getPlaylistWTrackVectorsRefined(25000, 5)

In [ ]:
#posFeatures, negFeatures = d.getPlaylistWTrackVectors(200000)

In [116]:
allFeatures = posFeatures + negFeatures

In [117]:
len(allFeatures)

40000

In [118]:
Y = [1 for i in range(len(posFeatures))] + [0 for i in range(len(negFeatures))]

In [119]:
from random import shuffle

In [120]:
shuffle(allFeatures)

In [121]:
c = list(zip(allFeatures, Y))
shuffle(c)
allFeatures, Y = zip(*c)
allFeatures = list(allFeatures)
Y = list(Y)

In [122]:
len(Y)

40000

In [123]:
negFeatures[0]

{'pos': 34,
 'playlist_collaborative': 'false',
 'playlist_description': 'nan',
 'playlist_duration_ms': 11532414,
 'playlist_modified_at': 1493424000,
 'playlist_name': 'Throwbacks',
 'playlist_num_albums': 47,
 'playlist_num_artists': 37,
 'playlist_num_edits': 6,
 'playlist_num_followers': 1,
 'playlist_num_tracks': 52,
 'danceability': 0.313,
 'energy': 0.553,
 'key': 9,
 'loudness': -3.101,
 'mode': 1,
 'speechiness': 0.0468,
 'acousticness': 0.000377,
 'instrumentalness': 0.229,
 'liveness': 0.181,
 'valence': 0.628,
 'tempo': 139.428,
 'duration_ms': 140177,
 'time_signature': 4,
 'popularity': 26,
 'release_date': '2014',
 'album_name': 'Dune Rats',
 'artist_name': 'Dune Rats',
 'track_name': 'Drugs'}

In [124]:
featuresDF = pd.DataFrame(allFeatures)

In [125]:
featuresDF.columns

Index(['acousticness', 'album_name', 'artist_name', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'playlist_collaborative', 'playlist_description',
       'playlist_duration_ms', 'playlist_modified_at', 'playlist_name',
       'playlist_num_albums', 'playlist_num_artists', 'playlist_num_edits',
       'playlist_num_followers', 'playlist_num_tracks', 'popularity', 'pos',
       'release_date', 'speechiness', 'tempo', 'time_signature', 'track_name',
       'valence'],
      dtype='object')

In [126]:
textCols = ['album_name', 'artist_name', 'playlist_description', 'playlist_name', 'track_name']
numericCols = ['acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode',
       'playlist_duration_ms', 'playlist_modified_at',
       'playlist_num_albums', 'playlist_num_artists', 'playlist_num_edits',
       'playlist_num_followers', 'playlist_num_tracks', 'popularity', 'pos',
       'release_date', 'speechiness', 'tempo', 'time_signature', 'valence']

In [127]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [128]:
featuresDF[numericCols].head(5)

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,playlist_duration_ms,...,playlist_num_edits,playlist_num_followers,playlist_num_tracks,popularity,pos,release_date,speechiness,tempo,time_signature,valence
0,0.01640,0.518,253735,0.813,0.748000,5,0.1380,-10.107,1,53442350,...,47,1,217,13.0,11,2013,0.0418,160.005,4,0.499
1,0.00217,0.271,428851,0.766,0.520000,5,0.0707,-8.030,1,20832783,...,32,3,78,62.0,34,1971,0.0329,142.914,4,0.803
2,0.76000,0.517,268722,0.422,0.000000,3,0.1720,-8.603,1,23679722,...,47,5,101,4.0,16,2013,0.0291,140.021,4,0.325
3,0.80700,0.717,117800,0.666,0.000001,2,0.0836,-15.732,1,1679085,...,2,1,12,20.0,60,2013,0.0283,97.680,4,0.961
4,0.00587,0.825,220626,0.832,0.000789,5,0.1140,-5.853,0,27551168,...,28,8,114,68.0,37,2009,0.0403,122.021,4,0.713


In [129]:
delNans = list(featuresDF.index[featuresDF[numericCols].isnull().T.any().T])
delNans

[15915, 17087, 37846, 38614]

In [130]:
featuresDF[numericCols].iloc[80240]

IndexError: single positional indexer is out-of-bounds

In [131]:
for i in range(len(delNans)-1, -1, -1):
    Y.pop(delNans[i])

In [132]:
featuresDF.iloc[0]

acousticness                                                         0.0164
album_name                Runner2: Future Legend of Rhythm Alien (The Or...
artist_name                                                    Gaijin Games
danceability                                                          0.518
duration_ms                                                          253735
energy                                                                0.813
instrumentalness                                                      0.748
key                                                                       5
liveness                                                              0.138
loudness                                                            -10.107
mode                                                                      1
playlist_collaborative                                                false
playlist_description                                                    nan
playlist_dur

In [133]:
delNans

[15915, 17087, 37846, 38614]

In [134]:
# drop rows with nan in them
featuresDF = featuresDF.drop(featuresDF.index[featuresDF[numericCols].isnull().T.any().T])

In [135]:
print(len(Y), len(featuresDF))

39996 39996


In [136]:
featuresDF[numericCols] = scaler.fit_transform(featuresDF[numericCols])

In [137]:
import nltk

In [138]:
def preprocess(t):
    t = t.lower()
    return nltk.word_tokenize(t)

In [139]:
for t in textCols:
    featuresDF[t] = featuresDF.apply(lambda row: preprocess(row[t]), axis=1)

In [140]:
featuresDF.head()

,acousticness,album_name,artist_name,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,...,playlist_num_followers,playlist_num_tracks,popularity,pos,release_date,speechiness,tempo,time_signature,track_name,valence
0,0.016466,"[runner2, :, future, legend, of, rhythm, alien...","[gaijin, games]",0.528571,0.049219,0.813,0.750251,0.454545,0.138138,0.774003,...,0.000000,0.865306,0.144444,0.044534,0.997522,0.043361,0.694108,0.8,[bit.trip],0.499
1,0.002179,"[led, zeppelin, iv]","[led, zeppelin]",0.276531,0.083188,0.766,0.521565,0.454545,0.070771,0.806224,...,0.000126,0.297959,0.688889,0.137652,0.976710,0.034129,0.619966,0.8,"[when, the, levee, breaks]",0.803
2,0.763052,"[the, madison, project]","[the, madison, project]",0.527551,0.052127,0.422,0.000000,0.272727,0.172172,0.797335,...,0.000253,0.391837,0.044444,0.064777,0.997522,0.030187,0.607416,0.8,"[colder, weather]",0.325
3,0.810241,"[the, essential, johnny, horton]","[johnny, horton]",0.731633,0.022851,0.666,0.000001,0.181818,0.083684,0.686741,...,0.000000,0.028571,0.222222,0.242915,0.997522,0.029357,0.423739,0.8,"[sal, 's, got, a, sugar, lip]",0.961
4,0.005894,"[the, fame, monster]","[lady, gaga]",0.841837,0.042797,0.832,0.000791,0.454545,0.114114,0.839996,...,0.000442,0.444898,0.755556,0.149798,0.995540,0.041805,0.529332,0.8,[telephone],0.713


In [141]:
from sklearn.preprocessing import MultiLabelBinarizer

for c in textCols:
    mlb = MultiLabelBinarizer()
    featuresDF = featuresDF.join(pd.DataFrame(mlb.fit_transform(featuresDF.pop(c)),
                              columns=mlb.classes_,
                              index=featuresDF.index).add_prefix(c+'_'))

In [142]:
featuresDF.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,playlist_collaborative,...,track_name_참고,track_name_첨엔,track_name_캐슬,track_name_타고,track_name_태연,track_name_피노키오,track_name_하나,track_name_헤어진,track_name_화창한,track_name_후에야
0,0.016466,0.528571,0.049219,0.813,0.750251,0.454545,0.138138,0.774003,1.0,false,...,0,0,0,0,0,0,0,0,0,0
1,0.002179,0.276531,0.083188,0.766,0.521565,0.454545,0.070771,0.806224,1.0,false,...,0,0,0,0,0,0,0,0,0,0
2,0.763052,0.527551,0.052127,0.422,0.000000,0.272727,0.172172,0.797335,1.0,false,...,0,0,0,0,0,0,0,0,0,0
3,0.810241,0.731633,0.022851,0.666,0.000001,0.181818,0.083684,0.686741,1.0,false,...,0,0,0,0,0,0,0,0,0,0
4,0.005894,0.841837,0.042797,0.832,0.000791,0.454545,0.114114,0.839996,0.0,false,...,0,0,0,0,0,0,0,0,0,0


In [145]:
featuresDF.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'playlist_collaborative',
       ...
       'track_name_참고', 'track_name_첨엔', 'track_name_캐슬', 'track_name_타고',
       'track_name_태연', 'track_name_피노키오', 'track_name_하나', 'track_name_헤어진',
       'track_name_화창한', 'track_name_후에야'],
      dtype='object', length=61886)

In [146]:
featuresDF = pd.get_dummies(featuresDF, columns=["playlist_collaborative"])

In [147]:
featuresDF.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,playlist_duration_ms,...,track_name_캐슬,track_name_타고,track_name_태연,track_name_피노키오,track_name_하나,track_name_헤어진,track_name_화창한,track_name_후에야,playlist_collaborative_false,playlist_collaborative_true
0,0.016466,0.528571,0.049219,0.813,0.750251,0.454545,0.138138,0.774003,1.0,0.639993,...,0,0,0,0,0,0,0,0,1,0
1,0.002179,0.276531,0.083188,0.766,0.521565,0.454545,0.070771,0.806224,1.0,0.244150,...,0,0,0,0,0,0,0,0,1,0
2,0.763052,0.527551,0.052127,0.422,0.000000,0.272727,0.172172,0.797335,1.0,0.278708,...,0,0,0,0,0,0,0,0,1,0
3,0.810241,0.731633,0.022851,0.666,0.000001,0.181818,0.083684,0.686741,1.0,0.011645,...,0,0,0,0,0,0,0,0,1,0
4,0.005894,0.841837,0.042797,0.832,0.000791,0.454545,0.114114,0.839996,0.0,0.325703,...,0,0,0,0,0,0,0,0,1,0


In [148]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(featuresDF, Y, test_size = 0.20)

In [155]:
import scipy
X_train_sparse = scipy.sparse.csr_matrix(X_train.values)

In [158]:
X_test_sparse = scipy.sparse.csr_matrix(X_test.values)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# create model
model = Sequential()
model.add(Dense(100, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 4096
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 2)

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
y_pred

In [ ]:
y_pred = (y_pred > 0.5)

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(Y_test, y_pred)

In [ ]:
acc

In [151]:
## SVMS

In [162]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train_sparse, Y_train)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [163]:
Y_test_preds = clf.predict(X_test_sparse)

In [157]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X_train_sparse, Y_train)  

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [159]:
Y_test_preds = logreg.predict(X_test_sparse)

In [164]:
Y_test_preds

array([0, 0, 0, ..., 0, 0, 0])

In [165]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_test_preds)

0.49725